In [11]:
import pandas as pd
import numpy as np
import os
import sys
import random

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов

In [14]:
dir_path = "C:/work/licenses"
if len(os.listdir(dir_path)) != 2:
    print("В целевой папке должны быть только актуальный Реестр МО и файл лицензий")
    sys.exit()
for file_name in os.listdir(dir_path):
    if file_name.split("_")[0] == "1.2.643.5.1.13.13.11.1461":
        ref_parh = dir_path + "/" + file_name
    else:
        lic_part = dir_path + "/" + file_name
df_ref = pd.read_excel(ref_parh, sheet_name="Справочник", index_col=None, dtype="str")


In [57]:
df_lic = pd.read_excel(lic_part, sheet_name="Лист 1", index_col=None, dtype="str")
df_lic = df_lic.drop(
    index=range(0, df_lic[df_lic["Unnamed: 1"] == "Дата"].index[0])
).reset_index(drop=True)
df_lic.columns = df_lic.iloc[0]
df_lic = df_lic.drop(index=0).reset_index(drop=True)

In [75]:
df_lic_s = df_lic[["№", "ИНН", "ОГРН"]]
med_name_list = []
for name in df_ref["medicalSubjectName"].unique():
    if "едицинская" in name:
        med_name_list.append(name)
med_name_list
df_ref_s = df_ref[
    (df_ref["medicalSubjectName"].isin(med_name_list)) & (df_ref["deleteDate"].isnull())
][
    [
        "oid",
        "nameFull",
        "nameShort",
        "inn",
        "ogrn",
        "regionName",
        "moDeptName",
        "parentId",
    ]
]

In [129]:
df_lic_inn = df_lic_s.merge(df_ref_s, left_on="ИНН", right_on="inn", how="left")
df_lic_inn["source"] = "inn"
df_lic_ogrn = df_lic_s.merge(df_ref_s, left_on="ОГРН", right_on="ogrn", how="left")
df_lic_ogrn["source"] = "ogrn"
df_lic_result = pd.concat([df_lic_inn, df_lic_ogrn]).sort_values(by="№").reset_index(drop=True)

In [134]:
for lic_num in df_lic_result["№"].unique():
    df_lic_0 = df_lic_result[df_lic_result["№"] == lic_num]
    for oid_num in df_lic_0["oid"].unique():
        df_lic_1 = df_lic_0[df_lic_0["oid"] == oid_num]
        if df_lic_1.shape[0] == 2:
            df_lic_result.loc[
                df_lic_1.index[0] : df_lic_1.index[-1], "source"
            ] = "inn&ogrn"
df_lic_result.drop_duplicates(inplace=True)

In [142]:
df_lic_result

,№,ИНН,ОГРН,oid,nameFull,nameShort,inn,ogrn,regionName,moDeptName,parentId,source
0,Л041-00110-01/00357877,0105025965,1020100703842,1.2.643.5.1.13.13.12.2.1.32,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,ГБУЗРА АРКБ,0105025965,1020100703842,Республика Адыгея (Адыгея),Органы исполнительной власти субъектов Российс...,NaN,inn&ogrn
2,Л041-00110-01/00361707,0105014177,1020100698595,1.2.643.5.1.13.13.12.3.1.3434,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,"ФГБОУ ВО ""МГТУ"", МГТУ, МАЙКОПСКИЙ ГОСУДАРСТВЕН...",0105014177,1020100698595,Республика Адыгея (Адыгея),Министерство образования и науки Российской Фе...,NaN,inn&ogrn
4,Л041-00110-01/00571612,0105047430,1060105011361,1.2.643.5.1.13.13.12.2.1.15354,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ АДЫГЕЯ""",0105047430,1060105011361,Республика Адыгея,Министерство внутренних дел Российской Федерации,NaN,inn&ogrn
6,Л041-00110-02/00323564,0275006624,1020202776109,1.2.643.5.1.13.13.12.2.2.17875,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ БАШКОРТОСТАН""",0275006624,1020202776109,Республика Башкортостан,Министерство внутренних дел Российской Федерации,NaN,inn&ogrn
8,Л041-00110-02/00340505,0274064870,1030204207582,1.2.643.5.1.13.13.12.2.2.11156,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ НАУЧНОЕ ...,УФИЦ РАН,0274064870,1030204207582,Республика Башкортостан,Министерство образования и науки Российской Фе...,NaN,inn&ogrn
...,...,...,...,...,...,...,...,...,...,...,...,...
16126,Л041-01197-26/00574277,2605010625,1022602421863,1.2.643.5.1.13.13.12.2.26.2381,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,"ГБУЗ СК ""БЛАГОДАРНЕНСКАЯ РБ""",2605010625,1022602421863,Ставропольский край,Органы исполнительной власти субъектов Российс...,NaN,inn&ogrn
16128,Л041-01197-26/00574400,2635221863,1162651069680,1.2.643.5.1.13.13.12.2.26.9800,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,"ГБУЗ СК ""СККПЦ № 1""",2635221863,1162651069680,Ставропольский край,Органы исполнительной власти субъектов Российс...,NaN,inn&ogrn
16130,Л041-01197-26/00574633,2618010703,1022600962240,1.2.643.5.1.13.13.12.2.26.2433,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,"ГБУЗ СК ""ПРЕДГОРНАЯ РБ""",2618010703,1022600962240,Ставропольский край,Органы исполнительной власти субъектов Российс...,NaN,inn&ogrn
16132,Л041-01197-26/00639943,2634110663,1222600010005,1.2.643.5.1.13.13.12.2.26.65938,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МЕДС...","ООО ""МЕДСПРАВКА""",2634110663,1222600010005,Ставропольский край,NaN,NaN,inn&ogrn


In [141]:
df_lic_result[df_lic_result["№"] == "Л041-00110-77/00575082"]

,№,ИНН,ОГРН,oid,nameFull,nameShort,inn,ogrn,regionName,moDeptName,parentId,source
2832,Л041-00110-77/00575082,7728095113,1027739054420,1.2.643.5.1.13.13.12.2.77.10443,Обособленное структурное подразделение Российс...,РДКБ ФГАОУ ВО РНИМУ им. Н.И. Пирогова,7728095113,1027739054420,г. Москва,Министерство здравоохранения Российской Федерации,NaN,inn&ogrn
2833,Л041-00110-77/00575082,7728095113,1027739054420,1.2.643.5.1.13.13.12.2.77.10614,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,"ФГБУ ""МНИИ педиатрии и детской хирургии"" Минзд...",7728095113,1027739054420,г. Москва,Министерство здравоохранения Российской Федерации,NaN,inn&ogrn
2834,Л041-00110-77/00575082,7728095113,1027739054420,1.2.643.5.1.13.13.12.2.77.86608,РОССИЙСКАЯ ДЕТСКАЯ КЛИНИЧЕСКАЯ БОЛЬНИЦА - ФИЛИ...,ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕННОГО АВТОНОМНО...,7728095113,1027739054420,г. Москва,Министерство здравоохранения Российской Федерации,1.2.643.5.1.13.13.12.2.77.8492,inn&ogrn
2835,Л041-00110-77/00575082,7728095113,1027739054420,1.2.643.5.1.13.13.12.2.77.10630,"ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УЧРЕЖДЕНИЕ ""РОССИЙ...",ФГУ РГНКЦ Росздрава,7728095113,1027739054420,г. Москва,Министерство здравоохранения Российской Федерации,NaN,inn&ogrn
2837,Л041-00110-77/00575082,7728095113,1027739054420,1.2.643.5.1.13.13.12.2.77.8492,Федеральное государственное автономное образов...,ФГАОУ ВО РНИМУ им. Н.И. Пирогова Минздрава России,7728095113,1027739054420,г. Москва,Министерство здравоохранения Российской Федерации,NaN,inn&ogrn


In [140]:
df_lic_result_light = df_lic_result.drop(["ИНН", "ОГРН"], axis=1)
df_lic_result_light

,№,oid,nameFull,nameShort,inn,ogrn,regionName,moDeptName,parentId,source
0,Л041-00110-01/00357877,1.2.643.5.1.13.13.12.2.1.32,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,ГБУЗРА АРКБ,0105025965,1020100703842,Республика Адыгея (Адыгея),Органы исполнительной власти субъектов Российс...,NaN,inn&ogrn
2,Л041-00110-01/00361707,1.2.643.5.1.13.13.12.3.1.3434,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,"ФГБОУ ВО ""МГТУ"", МГТУ, МАЙКОПСКИЙ ГОСУДАРСТВЕН...",0105014177,1020100698595,Республика Адыгея (Адыгея),Министерство образования и науки Российской Фе...,NaN,inn&ogrn
4,Л041-00110-01/00571612,1.2.643.5.1.13.13.12.2.1.15354,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ АДЫГЕЯ""",0105047430,1060105011361,Республика Адыгея,Министерство внутренних дел Российской Федерации,NaN,inn&ogrn
6,Л041-00110-02/00323564,1.2.643.5.1.13.13.12.2.2.17875,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ БАШКОРТОСТАН""",0275006624,1020202776109,Республика Башкортостан,Министерство внутренних дел Российской Федерации,NaN,inn&ogrn
8,Л041-00110-02/00340505,1.2.643.5.1.13.13.12.2.2.11156,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ НАУЧНОЕ ...,УФИЦ РАН,0274064870,1030204207582,Республика Башкортостан,Министерство образования и науки Российской Фе...,NaN,inn&ogrn
...,...,...,...,...,...,...,...,...,...,...
16126,Л041-01197-26/00574277,1.2.643.5.1.13.13.12.2.26.2381,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,"ГБУЗ СК ""БЛАГОДАРНЕНСКАЯ РБ""",2605010625,1022602421863,Ставропольский край,Органы исполнительной власти субъектов Российс...,NaN,inn&ogrn
16128,Л041-01197-26/00574400,1.2.643.5.1.13.13.12.2.26.9800,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,"ГБУЗ СК ""СККПЦ № 1""",2635221863,1162651069680,Ставропольский край,Органы исполнительной власти субъектов Российс...,NaN,inn&ogrn
16130,Л041-01197-26/00574633,1.2.643.5.1.13.13.12.2.26.2433,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,"ГБУЗ СК ""ПРЕДГОРНАЯ РБ""",2618010703,1022600962240,Ставропольский край,Органы исполнительной власти субъектов Российс...,NaN,inn&ogrn
16132,Л041-01197-26/00639943,1.2.643.5.1.13.13.12.2.26.65938,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МЕДС...","ООО ""МЕДСПРАВКА""",2634110663,1222600010005,Ставропольский край,NaN,NaN,inn&ogrn
